In [1]:
# Import packages
import torchvision 
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import torchvision.transforms as transforms


In [2]:
# Download MNIST data
# Transform the data to torch tensors and normalize it 
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307), ((0.3081)))])

trainset = torchvision.datasets.MNIST(root ='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root ='./data', train=False, download=True, transform=transform)

#image = trainset[5]
#plt.imshow(image)

# Prepare dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle =True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle =False, num_workers=4)

# Explore trainloader and testloader
print(trainloader.dataset.data.shape)
print(testloader.dataset.data.shape)


examples = iter(trainloader)
example_data, example_targets = next(examples)
#print(example_data)
#print(example_targets)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


The next step is building your CNN. The convolutional neural network is going to have 2 convolutional layers, each followed by a ReLU nonlinearity, and a fully connected layer. Each pooling layer halves both the height and the width of the image, so by using 2 pooling layers, the height and width are 1/4 of the original sizes. MNIST images have shape (1, 28, 28).
We are going to instanstiate two convolutional filters: the first one should have 5 channels, while the second one should have 10 channels. The kernel_size for both of them should be 3.

In [3]:
''' Create the CNN '''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Instantiate two convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, padding=1)
        # Instantiate the ReLU nonlinearity
        self.relu = nn.ReLU()
        # Instantiate a max pooling layer
        self.pool = nn.MaxPool2d(2,2)
        # Instantiate a fully-connected layer
        self.fc = nn.Linear(7*7*10, 10)
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 7*7*10)
        x = self.fc(x)
        return x 
        

In [4]:
''' Train the CNN '''
# Define the network
model = Net()

# Define the loss fct
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=3e-4)

for epoch in range(5):
    for batch_idx, (images , labels) in enumerate(trainloader):
        # Apply the model
        outputs = model(images)
        
        # Initiate optimizer to zero*
        optimizer.zero_grad()
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Calculate the Gradient
        loss.backward()
        
        # Optimize the weights
        optimizer.step()
        
        
    

In [5]:
''' Make predictions '''
# Evaluation mode
model.eval()

predictions = []
total, correct = 0, 0
for batch_inx, (images, labels) in enumerate(testloader):
    outputs = model(images)
    
    # Argmax the results of the net
    _, predicted = torch.max(outputs.data, 1)
    predictions.append(predicted)  
    
    # Calculate the test set accuracy
    correct += (predicted == labels).sum().item() 
    total += labels.size(0)
    
print('The testing accuracy is ', 100* correct /total)    
    
    

The testing accuracy is  97.84
